In [15]:
import requests
import json

/home/enbrands/miniconda3/envs/xiao/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.2.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


# 测试话术存储模块

In [18]:
url = "http://121.201.110.83:30504/sales_template_save"
post_data = {
    "industry_id": "默认",
    "template_id": "默认",
    "template_content": "默认销售"
}
json_post_data = json.dumps(post_data)
response = requests.post(url, data=json_post_data)


In [19]:
response.json()

{'status': 200, 'msg': 'success'}

# 测试模型回复模块

In [21]:
url = "http://121.201.110.83:30504/model_reply"
post_data = {
    "industry_id": "医美",
    "template_id": "project_1",
    "chat_id": "test_1",
    "user_input": "想问问皮肤相关的有啥项目"
}
json_post_data = json.dumps(post_data)
response = requests.post(url, data=json_post_data)


In [22]:
response.json()

{'status': 200,
 'msg': 'Model reply success',
 'model_reply': '{"customer_intention": "客户想了解皮肤相关的医美项目", "reply": "您好！很高兴您对皮肤相关的项目感兴趣。我们提供多种皮肤护理和改善的方案，比如水光针、激光美容、化学换肤、微针美容等。每个项目都有其独特的效果和适应人群，您可以告诉我您的具体需求，比如想改善的皮肤问题，我会为您推荐最合适的项目。期待您的回复！"}',
 'input_tokens': 256,
 'output_tokens': 111}

In [11]:
import json
a = '{"customer_intention": "", "reply": "ds"}'
c = json.loads(a)

In [14]:
c['reply']

'ds'

In [5]:
import gradio as gr
# 自定义 CSS
custom_css = """
#confirm-modal {
    display: none;
    position: fixed;
    z-index: 1000;
    left: 0;
    top: 0;
    width: 100%;
    height: 100%;
    background-color: rgba(0,0,0,0.5);
}

.modal-content {
    background-color: white;
    margin: 15% auto;
    padding: 20px;
    border-radius: 8px;
    width: 80%;
    max-width: 500px;
}

.modal-buttons {
    margin-top: 20px;
    text-align: right;
    gap: 8px;
    display: flex;
    justify-content: flex-end;
}
"""

# 自定义 JavaScript
custom_js = """
function showConfirmModal(industry_id, template_id) {
    const modal = document.getElementById('confirm-modal');
    const confirmText = document.getElementById('confirm-text');
    confirmText.textContent = `确定要保存话术模版到:\n行业id: ${industry_id}\n模板id: ${template_id}吗？`;
    modal.style.display = 'block';
}

function hideConfirmModal() {
    document.getElementById('confirm-modal').style.display = 'none';
}

// 点击模态框外部时关闭
window.onclick = function(event) {
    const modal = document.getElementById('confirm-modal');
    if (event.target === modal) {
        hideConfirmModal();
    }
}
"""

# 添加模态框的 HTML
modal_html = """
<div id="confirm-modal">
    <div class="modal-content">
        <h3>确认保存</h3>
        <p id="confirm-text"></p>
        <div class="modal-buttons">
            <button onclick="gradioSubmit(event)" id="confirm-yes">确认</button>
            <button onclick="hideConfirmModal()" id="confirm-no">取消</button>
        </div>
    </div>
</div>
"""

with gr.Blocks(css=custom_css, js=custom_js) as demo1:
    gr.HTML(modal_html)
    
    # ... 其他界面元素 ...
    
    with gr.Row():
        with gr.Column(scale=1):
            save_btn = gr.Button("💾存入数据库", elem_id="save-btn")
            
            # 保存确认函数
            def save_with_confirmation():
                return gr.Javascript(f'showConfirmModal("industry_id", "template_id")')
            
            # 绑定事件
            save_btn.click(
                fn=save_with_confirmation,
                inputs=None,
                outputs=None
            )
demo1.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/opt/anaconda3/lib/python3.11/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.38.1, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


In [6]:
demo1.close()

Closing server running on port: 7860


In [1]:
import gradio as gr

def welcome(name):
    return f"Welcome to Gradio, {name}!"

js = """
function createGradioAnimation() {
    var container = document.createElement('div');
    container.id = 'gradio-animation';
    container.style.fontSize = '2em';
    container.style.fontWeight = 'bold';
    container.style.textAlign = 'center';
    container.style.marginBottom = '20px';

    var text = 'Welcome to Gradio!';
    for (var i = 0; i < text.length; i++) {
        (function(i){
            setTimeout(function(){
                var letter = document.createElement('span');
                letter.style.opacity = '0';
                letter.style.transition = 'opacity 0.5s';
                letter.innerText = text[i];

                container.appendChild(letter);

                setTimeout(function() {
                    letter.style.opacity = '1';
                }, 50);
            }, i * 250);
        })(i);
    }

    var gradioContainer = document.querySelector('.gradio-container');
    gradioContainer.insertBefore(container, gradioContainer.firstChild);

    return 'Animation created';
}
"""
with gr.Blocks(js=js) as demo:
    inp = gr.Textbox(placeholder="What is your name?")
    out = gr.Textbox()
    inp.change(welcome, inp, out)

demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/opt/anaconda3/lib/python3.11/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.38.1, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


In [2]:
demo.close()

Closing server running on port: 7860
